In [1]:
import os
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
# Initialize the Groq model
model = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0.7,          # Adjust for response randomness
)

In [20]:
class BLOGState(TypedDict):

    title: str
    outline: str
    content: str
    evaluate: int

In [12]:
def create_outline(state: BLOGState) -> BLOGState:

    title = state['title']

    # Call LLM to generate outline
    prompt = f'Generate a detailed outline for a blog on the topic {title}'
    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [13]:
def create_blog(state: BLOGState) -> BLOGState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline} '

    content = model.invoke(prompt).content

    state['content'] = content 

    return state

In [26]:
def create_score(state: BLOGState) -> BLOGState:

    outline = state['outline']
    content = state['content']

    prompt = f"Evaluate the blog content against the provided outline. Rate it on a scale of 0 to 100 based on how well it follows the outline's structure and covers all key points. Return only an integer score (0-100), nothing else.\n\nOutline:\n{outline}\n\nBlog:\n{content}"
    
    evaluate = model.invoke(prompt).content

    try:
        score = int(evaluate.strip())
        if 0 <= score <= 100:
            state['evaluate'] = score
        else:
            state['evaluate'] = 0  # Fallback if score is out of range
    except ValueError:
        state['evaluate'] = 0  # Fallback if response is not an integer

    return state

In [27]:
graph = StateGraph(BLOGState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('create_score', create_score)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'create_score')
graph.add_edge('create_score', END)

workflow = graph.compile()

In [30]:
initial_state = {'title': 'The Benefits of Daily Meditation'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'The Benefits of Daily Meditation', 'outline': 'Here is a detailed outline for a blog on the benefits of daily meditation:\n\n**Title:** "Finding Inner Peace: The Benefits of Daily Meditation"\n\n**Introduction**\n\n* Brief overview of meditation and its increasing popularity\n* Explanation of the purpose of the blog post: to explore the benefits of daily meditation\n* Thesis statement: Incorporating daily meditation into your routine can have a significant impact on both physical and mental well-being, leading to a more balanced and fulfilling life.\n\n**I. Reduces Stress and Anxiety**\n\n* Explanation of how meditation affects the body\'s stress response\n* Discussion of the scientific evidence supporting meditation\'s ability to reduce stress and anxiety\n* Personal anecdotes or examples of how daily meditation can help manage stress and anxiety\n* Tips for incorporating meditation into a daily routine to reduce stress and anxiety\n\n**II. Improves Mental Clarity and Focus

In [31]:
print(final_state['outline'])

Here is a detailed outline for a blog on the benefits of daily meditation:

**Title:** "Finding Inner Peace: The Benefits of Daily Meditation"

**Introduction**

* Brief overview of meditation and its increasing popularity
* Explanation of the purpose of the blog post: to explore the benefits of daily meditation
* Thesis statement: Incorporating daily meditation into your routine can have a significant impact on both physical and mental well-being, leading to a more balanced and fulfilling life.

**I. Reduces Stress and Anxiety**

* Explanation of how meditation affects the body's stress response
* Discussion of the scientific evidence supporting meditation's ability to reduce stress and anxiety
* Personal anecdotes or examples of how daily meditation can help manage stress and anxiety
* Tips for incorporating meditation into a daily routine to reduce stress and anxiety

**II. Improves Mental Clarity and Focus**

* Explanation of how meditation improves cognitive function and concentra

In [19]:
print(final_state['content'])

**The Rise of AI in 2025: Trends, Applications, and Future Implications**

As we navigate the complexities of modern life, it's becoming increasingly evident that Artificial Intelligence (AI) is playing a pivotal role in shaping our daily experiences. From intelligent personal assistants to self-driving cars, AI is transforming industries and revolutionizing the way we live and work. As we enter 2025, it's essential to discuss the current state of AI and its future prospects. In this blog, we'll delve into the latest trends, applications, and emerging technologies in AI, as well as its potential implications for the future. In 2025, AI is poised to revolutionize industries and transform the way we live and work, with significant advancements in areas such as natural language processing, computer vision, and machine learning.

**Current State of AI in 2025**

The current AI landscape in 2025 is characterized by rapid growth and increasing adoption across various industries. One of the m

In [32]:
print(final_state['evaluate'])

98
